In [1]:
DEBUG = False
N_DEBUG = 100

In [2]:
import pandas as pd
import numpy as np
import transformers
import torch
import re
import sklearn
import spacy
nlp = spacy.load("en_core_web_sm")

In [3]:
test = pd.read_pickle("./dataset_test.pkl")

test = test[test["author"] == "human_answers"]
print("len(test_human)", len(test))
documents = test["answer"]
gold_labels = test["author"] == "human_answers" # convention: 0: machine, 1: human, see detector.py




#from gpt2outputdataset.detector_radford import DetectorRadford
#from detectgpt.detector_detectgpt import DetectorDetectGPT
from detector_guo import DetectorGuo
detector_classes = [DetectorGuo]#,DetectorRadford,DetectorDetectGPT]

from explainer_wrappers import LIME_Explainer, SHAP_Explainer
explainer_classes = [LIME_Explainer,SHAP_Explainer]

len(test_human) 152


In [4]:
from gpt2outputdataset.detector_radford import DetectorRadford


detector = DetectorRadford()

In [5]:
DEVICE = "cuda"
pattern = re.compile(r"<extra_id_\d+>")

base_model_name="facebook/opt-350m"
openai_model = False

cache_dir="./.cache"
# mask_model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model, cache_dir=cache_dir).to(DEVICE)
# mask_tokenizer = transformers.AutoTokenizer.from_pretrained(model, model_max_length=mask_model.config.n_positions, cache_dir=cache_dir)#.to(DEVICE)
do_top_k= False
do_top_p= False

In [6]:
base_model_kwargs = {}
if 'gpt-j' in base_model_name or 'neox' in base_model_name:
    base_model_kwargs.update(dict(torch_dtype=torch.float16))
if 'gpt-j' in base_model_name:
    base_model_kwargs.update(dict(revision='float16'))
base_model = transformers.AutoModelForCausalLM.from_pretrained(base_model_name, **base_model_kwargs, cache_dir=cache_dir).to(DEVICE)


optional_tok_kwargs = {"additional_special_tokens": ["<|loris|>"]}
# if "facebook/opt-" in base_model_name:
#     print("Using non-fast tokenizer for OPT")
#     optional_tok_kwargs['fast'] = False

base_tokenizer = transformers.AutoTokenizer.from_pretrained(base_model_name, **optional_tok_kwargs, cache_dir=cache_dir, padding_side='left')
base_tokenizer.pad_token_id = base_tokenizer.eos_token_id # TODO WHY??? 


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
predictions = None

In [8]:
decoded = None

In [9]:
if DEBUG and N_DEBUG > 0:
    documents = documents.iloc[0:N_DEBUG].to_list()

In [10]:
from tqdm import tqdm

In [11]:
label_flip_pairs = []

for document in tqdm(documents, desc="Generating perturbations"):
    np.random.seed(42)
    torch.manual_seed(42)
    doc = nlp(document)
    n_tokens_original = len(base_tokenizer(document, return_tensors="pt", padding=True).to(DEVICE).input_ids[0])
    original_prediction = detector.predict_label([document])[0]
    substrings = [''.join(token.text_with_ws for token in doc[:-i]) for i in range(1,len(document)) ]

    batch_size = 25
    for batch in (sklearn.utils.gen_batches(len(substrings), batch_size)):
        encoded = base_tokenizer(substrings[batch], return_tensors="pt", padding=True).to(DEVICE)
        sampling_kwargs = {}

        outputs = base_model.generate(**encoded, min_length=n_tokens_original-5, max_length=n_tokens_original+5, do_sample=True, **sampling_kwargs, pad_token_id=base_tokenizer.eos_token_id, eos_token_id=base_tokenizer.eos_token_id)
        decoded = base_tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions = detector.predict_label(decoded)
        
        if any(predictions != original_prediction):
            first_new_label = (predictions!=original_prediction).argmax(axis=0)

            #                        original  prompt                              first instance that flips label
            label_flip_pairs.append((document, substrings[batch][first_new_label], decoded[first_new_label]))
            break


Generating perturbations: 100%|██████████| 7/7 [00:32<00:00,  4.57s/it]


In [12]:
label_flip_pairs

[('Use VTIVX. The "Target Retirement 2045" and "Target Retirement 2045 Trust Plus" are the same underlying fund, but the latter is offered through employers.  The only differences I see are the expense ratio and the minimum investment dollars.  But for the purposes of comparing funds, it should be pretty close. Here is the list of all of Vanguard\'s target retirement funds. Also, note that the "Trust Plus" hasn\'t been around as long, so you don\'t see the returns beyond the last few years.  That\'s another reason to use plain VTIVX for comparison. See also: Why doesn\'t a mutual fund in my 401(k) have a ticker symbol?',
  'Use VTIVX. The "Target Retirement 2045" and "Target Retirement 2045 Trust Plus" are the same underlying fund, but the latter is offered through employers.  The only differences I see are the expense ratio and the minimum investment dollars.  But for the purposes of comparing funds, it should be pretty close. Here is the list of all of Vanguard\'s target retirement f

In [13]:
explainer = SHAP_Explainer(detector)

In [14]:
from IPython.core.display import HTML

In [15]:
original, prompt, label_flip_example = label_flip_pairs[2]
print(prompt)
display(HTML(explainer.get_highlighted_text_HTML(original)))
display(HTML(explainer.get_highlighted_text_HTML(prompt)))
display(HTML(explainer.get_highlighted_text_HTML(label_flip_example)))

The forms get updated every year, and the software providers need to get approved by the IRS every year. "Form is not yet finalized" means that this year form hasn't been approved yet. IRS starts accepting returns on January 31st anyway, nothing to be worried about. Why are you nearing a deadline? The deadline for 1120 (corporate tax 


In [16]:
# basic idea: assert that exp(original)[original - prompt] <substantially different than> exp(label_flip_example)[label_flip_example - prompt]
# i.e. the new/changed section is assigned the opposite label (TODO hard coded: "machine") more often

In [17]:
label_flip_pairs

[('Use VTIVX. The "Target Retirement 2045" and "Target Retirement 2045 Trust Plus" are the same underlying fund, but the latter is offered through employers.  The only differences I see are the expense ratio and the minimum investment dollars.  But for the purposes of comparing funds, it should be pretty close. Here is the list of all of Vanguard\'s target retirement funds. Also, note that the "Trust Plus" hasn\'t been around as long, so you don\'t see the returns beyond the last few years.  That\'s another reason to use plain VTIVX for comparison. See also: Why doesn\'t a mutual fund in my 401(k) have a ticker symbol?',
  'Use VTIVX. The "Target Retirement 2045" and "Target Retirement 2045 Trust Plus" are the same underlying fund, but the latter is offered through employers.  The only differences I see are the expense ratio and the minimum investment dollars.  But for the purposes of comparing funds, it should be pretty close. Here is the list of all of Vanguard\'s target retirement f

In [18]:
import krippendorff

In [19]:
results = []
for explainer_class in explainer_classes:
        explainer = explainer_class(detector)
        for original, prompt, label_flip_example in label_flip_pairs:
            tokens_original_minus_prompt = explainer.tokenize(original)[len(explainer.tokenize(prompt)):]
            tokens_label_flip_minus_prompt = explainer.tokenize(label_flip_example)[len(explainer.tokenize(prompt)):]
            exp_original_minus_prompt = explainer.get_fi_scores(original, fill=True)[0][len(explainer.tokenize(prompt)):] # TODO hard coded: "machine"
            exp_label_flip_minus_prompt = explainer.get_fi_scores(label_flip_example, fill=True)[0][len(explainer.tokenize(prompt)):] # setting fill=True returns all features (not just the top_k) 
         
            exp_original_prompt_only = explainer.get_fi_scores(original, fill=True)[0][0:len(explainer.tokenize(prompt))]
            exp_label_flip_prompt_only = explainer.get_fi_scores(label_flip_example, fill=True)[0][0:len(explainer.tokenize(prompt))]
           # print(exp_original_minus_prompt)
           # print(exp_label_flip_minus_prompt)
            mean_fi_machine_label_flip = np.mean([fi_score for _, fi_score in exp_label_flip_minus_prompt])
            mean_fi_original = np.mean([fi_score for _, fi_score in exp_original_minus_prompt])

            cannonical_form = np.vstack([exp_original_prompt_only, exp_label_flip_prompt_only])
            k_alpha = krippendorff.alpha(cannonical_form, level_of_measurement="interval")
            results.append((explainer.__class__.__name__, k_alpha, mean_fi_original, mean_fi_machine_label_flip,))


In [20]:
df = pd.DataFrame(results, columns=["Explainer", "Krippendorf's alpha on prompt","E[original - prompt]", "E[lf - prompt]"])

In [21]:
df["Change original -> lf [%]"] = ((df["E[original - prompt]"] - df["E[lf - prompt]"] ) / df["E[original - prompt]"]) * 100
df

,Explainer,Krippendorf's alpha on prompt,E[original - prompt],E[lf - prompt],Change original -> lf [%]
0,LIME_Explainer,0.593404,-0.000021,0.000000,100.000000
1,LIME_Explainer,0.595069,-0.001718,0.000429,124.950972
2,LIME_Explainer,0.590053,-0.003674,0.001796,148.882905
3,LIME_Explainer,0.594465,0.000000,0.000000,NaN
4,LIME_Explainer,0.592731,-0.002221,0.002476,211.496820
5,LIME_Explainer,0.589260,-0.003574,-0.001891,47.073857
6,LIME_Explainer,0.577914,-0.005886,0.028169,578.603391
7,SHAP_Explainer,0.595014,-0.001848,-0.043575,-2257.702122
8,SHAP_Explainer,0.596018,-0.005580,0.006347,213.756153
9,SHAP_Explainer,0.591816,-0.008718,0.011759,234.871303


In [22]:
df.groupby("Explainer").mean()

,Krippendorf's alpha on prompt,E[original - prompt],E[lf - prompt],Change original -> lf [%]
Explainer,,,,
LIME_Explainer,0.590414,-0.002442,0.004426,201.834657
SHAP_Explainer,0.592270,-0.005988,0.001043,-152.445194


In [23]:
np.mean([fi_score for _, fi_score in exp_label_flip_minus_prompt])

0.01393137565658738

In [24]:
exp_label_flip_minus_prompt

[(33, -0.03665674431249499),
 (34, -0.01942520192824304),
 (35, -0.022524658357724547),
 (36, 0.0023456146009266376),
 (37, -0.01859639724716544),
 (38, 0.00733706820756197),
 (39, 0.029099322855472565),
 (40, 0.011824396904557943),
 (41, -1.3148877769708633e-05),
 (42, 0.0017422079108655453),
 (43, -0.015610974747687578),
 (44, 0.016743616371128995),
 (45, 0.004950770172469605),
 (46, 0.002366626165055025),
 (47, 0.0017224231709499294),
 (48, 0.14037172314457189),
 (49, 0.20282264084417898),
 (50, 0.04601623397149937),
 (51, 0.1133311263967012),
 (52, -0.0390842378855045),
 (53, -0.03371355449951394),
 (54, -0.02195333596489532),
 (55, -0.03784894664011104),
 (56, -0.000893554496732387)]